# Ticks - data cleaning

In [1]:
import numpy as np
from pandas import read_csv, concat, Series
from helpers.data_cleaning import score_and_add_columns, score
from helpers.display import pokaz_zestawienie

Max number of points in the tick knowledge test.

In [2]:
max_point_n = 29

## Load and parse data

Merging two CSV files (with and without question with tick images) with answers into one.

Due to technical limitation the first questionnaire version did not display the question with ticks photos. It was corrected in the second questionnaire version and all survey answers were analyzed without this question.

In [3]:
without_photo = read_csv("data/tick_survey_2018_without_images.csv")
without_photo = without_photo.assign(with_images=False)
with_photo = read_csv("data/tick_survey_2018_with_images.csv")
with_photo = with_photo.assign(with_images=True)
# odpowiedzi is Polish for answers
odpowiedzi = concat([without_photo, with_photo]).reset_index(drop=True)
odpowiedzi.head(0).T

""
Timestamp
Płeć
Ile masz lat? (skończone)
Miejsce zamieszkania
Czy jesteś członkiem organizacji harcerskiej na terenie Polski?
Do jakiej organizacji harcerskiej należysz?
Od ilu lat jesteś członkiem organizacji harcerskiej?
Czy pełnisz funkcję instruktorską (np. drużynowy) lub jesteś instruktorem?
W ilu obozach uczestniczyłaś/-łeś?
"Zaznacz wszystkie obrazki, które przedstawiają kleszcza"


In [4]:
odpowiedzi_rows_n = len(odpowiedzi)
f'Number of rows before cleaning: {odpowiedzi_rows_n}'

'Number of rows before cleaning: 3623'

## Change column names

In [5]:
names_map = {
    'Ile masz lat? (skończone)': 'age',
    'Płeć': 'sex',
    'Miejsce zamieszkania': 'place_of_residence',
    'Do jakiej organizacji harcerskiej należysz?': 'scouting_organization',
    'Czy jesteś członkiem organizacji harcerskiej na terenie Polski?': 'is_polish_scout',
    'Czy pełnisz funkcję instruktorską (np. drużynowy) lub jesteś instruktorem?': 'is_instructor',
    'W ilu obozach uczestniczyłaś/-łeś?': 'camps_count',
    'Od ilu lat jesteś członkiem organizacji harcerskiej? ': 'years_in_scouting',
    'Zaznacz wszystkie obrazki, które przedstawiają kleszcza': 'tick_picture',
    'Z poniższej listy wybierz choroby roznoszone przez kleszcze': 'TBD',
    'Z poniższej listy wybierz objawy, które często występują przy chorobach odkleszczowych': 'TBD_symptoms',
    'Jakie będzie Twoje postępowanie w przypadku ugryzienia przez kleszcza?': 'po_ugryzieniu',
    'Co zrobić w przypadku pojawienia się objawów po ugryzieniu przez kleszcza? (np. zaczerwienienia, gorączki, rumienia)': 'after_bite',
    'Jak wyjąć kleszcza?': 'how_to_remove',
    'Jakie profilaktyczne działania możesz podjąć, aby zminimalizować ryzyko ugryzienia przez kleszcza?': 'profilaktyka',
    'Gdzie żyją kleszcze?': 'where_ticks_live',
    'Kleszcze można wykręcać tylko w jedną, określoną stronę': 'MIT_spin',
    'Przed wyjęciem kleszcza należy go  posmarować masłem, olejem lub spirytusem': 'MIT_butter',
    'Z każdym kleszczem należy jechać na pogotowie/SOR, aby lekarz go wyciągnął - nie można go samemu wyjmować': 'MIT_SOR',
    'Przy boreliozie zawsze pojawia się rumień wędrujący': 'MIT_always_rumien',
    'Tylko duże kleszcze roznoszą choroby': 'MIT_big_tick',
    'Należy poczekać aż kleszcz sam wyjdzie ze skóry': 'MIT_tick_go_away',
    'Aby wyciągnąć kleszcza łapiemy kleszcza za odwłok by porazić jego układ nerwowy i trzymamy aż sam wyjdzie': 'MIT_paralize_tick',
    'Jak szybko wyjmę kleszcza to na pewno nie zachoruję': 'MIT_run_tick_run',
    'Aby wykręcić kleszcza wystarczy koliście łaskotać go wacikiem aż sam wyjdzie': 'MIT_tickles',
    'Rumień zawsze pojawia się w miejscu ugryzienia': 'MIT_place_of_bite',
    'Istnieje szczepionka przeciwko kleszczom': 'MIT_vaccine',
    'Kleszcze występują tylko w lasach': 'MIT_forest_only',
    'Nieprawidłowe wyjęcie kleszcza zwiększa ryzyko zarażenia': 'MIT_wrong_remove',
    'Na boreliozę choruje się od razu po ugryzieniu': 'MIT_speed_ill',
    'Preparaty do odstraszania kleszczy (np. spraye, obroże, krople, kadzidła itp.) chronią mnie w 100% przed ugryzieniem przez kleszcza': 'MIT_scarry_tick',
    'Kleszcze spadają na swoją ofiarę z drzewa, przynajmniej z jednego metra': 'MIT_tick_jump',
    'Borelioza może pojawić się dopiero po pewnym czasie od ugryzienia': 'MIT_waiting_boreliosis',
    'Jeśli nie wystąpi rumień wędrujący po ugryzieniu przez kleszcza to znaczy, że jestem zdrowa/y': 'MIT_safe',
    'Kleszcza można „złapać” tylko latem': 'MIT_summer_tick',
    'Jak kawałek kleszcza zostanie w ciele, to nie trzeba się przejmować ani podejmować żadnych działań w kierunku usunięcia jego resztek': 'MIT_part_tick',
    'Kleszcza można „złapać” w mieście': 'MIT_city',
    'Kleszcza należy wykręcić poprzez wykonywanie palcem kolistych ruchów po skórze wokół kleszcza': 'MIT_clock',
    'Szacunkowo ile tygodni spędzasz w ciągu roku na łonie natury (w lesie, parku, na łące itp.)?': 'time_in_nature',
    'Ile razy byłaś/łeś ugryziony kiedyś przez kleszcza?': 'tick_bites',
    'Skąd czerpiesz wiedzę na temat kleszczy i chorób które roznoszą?': 'source_of_knowledge',
    'Czy uczestniczyłaś/łeś w zajęciach z pierwszej pomocy?': 'first_aid_course',
    'Czy podczas zajęć z pierwszej pomocy był poruszany temat kleszczy np. chorób odkleszczowych lub co zrobić w przypadku ugryzienia przez kleszcza?': 'first_aid_content',
    'Jakie urządzenia do usuwania kleszczy (spośród dostępnych na polskim rynku) widziałaś/łeś lub miałaś/łeś okazję używać?': 'tick_removal',
}
all_answers = odpowiedzi.rename(columns=names_map)

## Filter by age

In [6]:
accepted_ages = list(range(16, 22))
accepted_ages

[16, 17, 18, 19, 20, 21]

In [7]:
age_filtered_answers = all_answers.query('age in @accepted_ages').copy()
len(age_filtered_answers)

3385

Number and percent of discarded answers due to inapropriate age:

In [8]:
discarded_due_to_age = len(all_answers) - len(age_filtered_answers)
discarded_due_to_age, discarded_due_to_age / len(all_answers) * 100

(238, 6.569141595362959)

For brevity:

In [9]:
a = age_filtered_answers

## Convert categorical columns

In [10]:
a.is_polish_scout = (a.is_polish_scout == 'Tak')
a.is_instructor = (a.is_instructor == 'Tak')
a = a.assign(is_women=(a.sex == 'Kobieta'))

# 'Nie' means no, 'Tak' or desctiption means yes
a = a.assign(first_aid_course_participated=(a.first_aid_course != 'Nie'))

In [11]:
residence_place_sizes = {
    'Wieś': 1,
    'Miasto do 50 tys. mieszkańców': 2,
    'Miasto od 50 do 100 tys. mieszkańców': 3,
    'Miasto powyżej 100 tys. mieszkańców': 4 
}

a = a.assign(
    size_of_residence_place=a.place_of_residence.map(residence_place_sizes)
)

## Verify time spent outside

In [12]:
time_in_nature_threshold = 52   # there are 52 weeks in a year, no more

print(f'Clamping number of weeks in nature to {time_in_nature_threshold}, as some ppl have entered')
print(f'ridiculus values as {max(a["time_in_nature"])} days which were skewing the data distribution')

a = a.assign(time_in_nature_pruned=Series(
    (value if value <= time_in_nature_threshold else np.NaN)
    for value in a['time_in_nature']
).values)

a = a.assign(time_in_nature_clamped=Series(
    min(value, time_in_nature_threshold)
    for value in a['time_in_nature']
).values)

Clamping number of weeks in nature to 52, as some ppl have entered
ridiculus values as 12740171513718.0 days which were skewing the data distribution


In [13]:
a["time_in_nature"].sort_values(ascending=False).head(10)

875     1.274017e+13
3171    5.638474e+08
152     2.615282e+08
1444    5.677744e+06
2924    1.000000e+04
634     5.000000e+03
1132    6.660000e+02
2440    3.650000e+02
566     3.650000e+02
981     3.650000e+02
Name: time_in_nature, dtype: float64

## Verify number of tick bites

In [14]:
tick_bites = a['tick_bites']
bites_threshold = np.percentile(tick_bites, 99)
print(f'Clamping number of bites to 99th percentile ({bites_threshold}), as some ppl have')
print(f'entered ridiculus values as {max(tick_bites)} bites which were skewing the data distribution')

Clamping number of bites to 99th percentile (40.0), as some ppl have
entered ridiculus values as 28384482838485 bites which were skewing the data distribution


In [15]:
tick_bites.sort_values(ascending=False).head(10)

3171    28384482838485
1775          17362727
2022          10000000
1518           9999999
1810             47887
781               1233
1850               180
3517               150
3086               100
1264               100
Name: tick_bites, dtype: int64

In [16]:
print(f'MAD: {tick_bites.mad()}, median: {tick_bites.median()}, mean: {tick_bites.mean()} reported number of tick bites.')

MAD: 16765788747.909801, median: 1.0, mean: 8385382647.429837 reported number of tick bites.


In my Master thesis I used percentile to clear data from abnormally high numbers of tick bites. Now I decided to set cut-off point to 200 tick bites. It decreased the number of removed records to 6 people who reported tick bites of 1233 of more.

In [17]:
a.tick_bites = Series(
    (value if value <= 200 else np.NaN)
    for value in tick_bites
).values

## Discard incorrect answers

In [18]:
n_in_a_before_cleaning = len(a)
n_in_a_before_cleaning

3385

Exclude any invalid answers (i.e. having missing value in `tick_bites` or in `time_in_nature_pruned` columns - those are missing because I set those to be `NaN` in "Verify time spent outside" and "Verify number of tick bites" steps)- keeping high-confidence subset

In [19]:
a = a.dropna(how='any', subset=['tick_bites', 'time_in_nature_pruned'])

In [20]:
n_in_a_after_cleaning = len(a)
n_in_a_after_cleaning

3322

In [ ]:
(
    f'Number of answers discarded due to abnormally high tick bites and time '
    f'spent in nature: {n_in_a_before_cleaning - n_in_a_after_cleaning}'
)

'Number of answers discarded due to abnormally high tick bites and time spent in nature: 63'

## Other cleaning

Remove hooligan giving rude, inappropriate answers

In [ ]:
is_rude_answer = a.source_of_knowledge == 'z nauki roslinnej moim pokarmem sa kleszcze'
a = a[is_rude_answer == False]
a.head(2)

Change is_polish_scout to False for participant who stated True, but later in the questionnaire commented that they do not belong to any scout organization and they were 0 years_in_scouting.

In [ ]:
is_not_scout = a.scouting_organization == 'Zadna'
a.loc[is_not_scout, 'is_polish_scout'] = False

In [ ]:
a.years_in_scouting = a.years_in_scouting.replace(2008, 10)
a = a[~a.is_polish_scout | (a.camps_count <= 75)]
a = a.reset_index()

In [ ]:
all_answers = a

# Grading answers

In [ ]:
def score_tick_picture(wartosc, wiersz):
    correct_answers = ['1', '4']
    wrong_answers = ['2', '3']
    return score(wartosc, correct_answers, wrong_answers)

def score_TBD(wartosc, wiersz):
    correct_answers = ['Borelioza', 'Wirusowe zapalenie mózgu i opon mózgowo-rdzeniowych']
    wrong_answers = ['Ospa wietrzna', 'Żółtaczka', 'AIDS', 'Wścieklizna', 'Odra']
    return score(wartosc, correct_answers, wrong_answers)

def score_TBD_symptoms(wartosc, wiersz):
    correct_answers = ['Wysoka gorączka','Ból głowy', 'Rumień wędrujący', 'Bóle mięśniowo-stawowe', 'Wymioty']
    wrong_answers = ['Krwawa biegunka', 'Żółtaczka']
    return score(wartosc, correct_answers, wrong_answers)

def score_po_ugryzieniu(wartosc, wiersz):
    correct_answers = ['Wyjmę go', 'Poproszę o pomoc kogoś, kto wie jak wyjąć kleszcza']
    wrong_answers = ['Poczekam aż kleszcz sam wyjdzie', 'Natychmiast zgłoszę się do szpitala albo zadzwonię na 112']
    return score(wartosc, correct_answers, wrong_answers, any_correct_gives_full_points=True)

def score_after_bite(wartosc, wiersz):
    correct_answers = ['Powiem rodzicowi lub opiekunowi', 'Zgłoszę się do lekarza / pielęgniarki / ratownika medycznego']
    wrong_answers = ['Wezmę paracetamol albo ibuprofen', 'Poczekam parę dni aż samo przejdzie']
    return score(wartosc, correct_answers, wrong_answers, any_correct_gives_full_points=True)

def score_how_to_remove(wartosc, wiersz):
    correct_answers = ['Przy użyciu specjalnych narzędzi, np. pętelką, szczypcami, pęsetą. Po wyjęciu zdezynfekować miejsce ukąszenia.']
    wrong_answers = ['Nie wolno wyjmować, tylko lekarz może to zrobić', 'Należy przypalić kleszcza zapalniczką', 'Należy łaskotać kleszcza aż sam wyjdzie', 'Należy posmarować go masłem i czekać aż wyjdzie']
    return score(wartosc, correct_answers, wrong_answers)

def score_profilaktyka(wartosc, wiersz):
    #wywazyc pytanie
    correct_answers = ['Założyć ubrania w jasnych kolorach, dzięki czemu łatwiej będzie zobaczyć kleszcza', 'Założyć ubrania z długimi rękawami i nogawkami oraz wsadzić nogawki w skarpetki, założyć nakrycie na głowę', 'Spryskać się preparatami odstraszającymi kleszcze', 'Użyć opaski odstraszającej kleszcze', 'Unikać obszarów bytowania kleszczy']
    wrong_answers = ['Ubrać się na czarno, żeby kleszcze mnie nie widziały']
    return score(wartosc, correct_answers, wrong_answers)


def score_where_ticks_live(wartosc, wiersz):
    correct_answers = ['W parku miejskim', 'Na łące', 'W lesie', 'W ogrodzie', 'W pomieszczeniach zamkniętych, np. w urzędzie, w szkole']
    wrong_answers = [' ']
    return score(wartosc, correct_answers, wrong_answers, wrong_gets_same_weight_as_correct=True)

In [ ]:
scoring_functions = {
    'tick_picture': score_tick_picture,
    'TBD': score_TBD,
    'TBD_symptoms': score_TBD_symptoms,
    'po_ugryzieniu': score_po_ugryzieniu,
    'after_bite': score_after_bite,
    'how_to_remove': score_how_to_remove,
    'profilaktyka': score_profilaktyka,
    'where_ticks_live': score_where_ticks_live
}

In [ ]:
is_myth = {
    # myth_name-> if is myth (True -> myth/False -> not myth)
    'MIT_spin': True,
    'MIT_butter': True,
    'MIT_SOR': True,
    'MIT_always_rumien': True,
    'MIT_big_tick': True,
    'MIT_tick_go_away': True,
    'MIT_paralize_tick': True,
    'MIT_run_tick_run': True,
    'MIT_tickles': True,
    'MIT_place_of_bite': True,
    'MIT_vaccine': True,
    'MIT_forest_only': True,
    'MIT_wrong_remove': False,
    'MIT_speed_ill': True,
    'MIT_scarry_tick': True,
    'MIT_tick_jump': True,
    'MIT_waiting_boreliosis': False,
    'MIT_safe': True,
    'MIT_summer_tick': True,
    'MIT_part_tick': True,
    'MIT_city': False,
    'MIT_clock': True
}

g (graded_answers) is a dataframe with answers (a) with grades for correct answers to myth questions added.

In [ ]:
g, columns_with_scores, scored_columns = score_and_add_columns(scoring_functions, is_myth, a)

In [ ]:
g.head(2)

In [ ]:
columns_with_scores_without_images = list(
    set(columns_with_scores) - {'tick_picture_score'}
)

Using 30 columns to grade

In [ ]:
assert len(columns_with_scores) == 30

In [ ]:
g = g.assign(total_score_without_images=Series(
    float(sum(wiersz))
    for wiersz in g[columns_with_scores_without_images].itertuples(index=False)
).values)

g = g.assign(total_score_with_images=Series(
    float(sum(wiersz))
    for wiersz in g[columns_with_scores].itertuples(index=False)
).values)

## Scale to %

In [ ]:
g['score_as_percent'] = g.total_score_without_images / max_point_n * 100
g.head(2)

In [ ]:
g.to_csv('data/cleaned_answers.csv')

## List of columns

In [ ]:
odpowiedzi = list(scoring_functions.keys())
mity = [k for k in all_answers.keys() if k.startswith('MIT')]
pokaz_zestawienie(
    ogólne=[k for k in all_answers.keys() if k not in mity and k not in odpowiedzi],
    odpowiedzi=odpowiedzi, mity=mity,
)